# 서울시 편의점 매출 결정요인 분석
## 01. 데이터 전처리

---

### 연구 질문
**서울시 편의점 매출을 결정하는 핵심 요인은 무엇인가?**

### 연구 배경
편의점 시장이 포화 상태에 접어들면서, 신규 출점 시 입지 선정의 중요성이 높아지고 있다. 
단순히 "유동인구가 많은 곳"이 좋다는 막연한 기준이 아닌, 데이터에 기반한 의사결정이 필요하다.

### 연구 가설
- **H1**: 유동인구가 많을수록 매출이 높다
- **H2**: 점포수(밀집도)가 높을수록 매출이 높다  
- **H3**: 상권유형(골목/발달/전통시장/관광특구)에 따라 매출 차이가 있다

### 이 노트북의 목표
가설 검증에 필요한 변수들을 하나의 분석용 데이터셋으로 통합한다.

---

## 1. 환경 설정

In [12]:
import pandas as pd
import numpy as np
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:,.0f}'.format)

print("분석 환경 준비 완료")

분석 환경 준비 완료


## 2. 데이터 소스 확인

가설 검증에 필요한 변수:
- **종속변수**: 매출 → `sales.csv`
- **독립변수 1**: 유동인구 → `population.csv`
- **독립변수 2**: 점포수 → `stores.csv`
- **독립변수 3**: 상권유형 → `districts.csv`

In [13]:
# 데이터 로드
sales_df = pd.read_csv('../../data/processed/sales.csv', encoding='utf-8-sig')
store_df = pd.read_csv('../../data/processed/stores.csv', encoding='utf-8-sig')
pop_df = pd.read_csv('../../data/processed/population.csv', encoding='utf-8-sig')
area_df = pd.read_csv('../../data/processed/districts.csv', encoding='cp949')

print("데이터 로드 완료")
print(f"  매출: {sales_df.shape[0]:,}건")
print(f"  점포: {store_df.shape[0]:,}건")
print(f"  유동인구: {pop_df.shape[0]:,}건")
print(f"  상권영역: {area_df.shape[0]:,}건")

데이터 로드 완료
  매출: 256,378건
  점포: 528,758건
  유동인구: 11,475건
  상권영역: 1,650건


## 3. 분석 범위 설정

- **업종**: 편의점 (서비스_업종_코드 = 'CS300002')
- **기간**: 2022년 1분기 ~ 2025년 3분기 (15개 분기)
- **단위**: 행정동 × 분기

In [14]:
# 편의점만 필터링
cvs_sales = sales_df[sales_df['서비스_업종_코드'] == 'CS300002'].copy()
cvs_store = store_df[store_df['서비스_업종_코드'] == 'CS300002'].copy()

# 분석 기간 필터링 (2022Q1 ~ 2025Q3)
cvs_sales = cvs_sales[(cvs_sales['기준_년분기_코드'] >= 20221) & (cvs_sales['기준_년분기_코드'] <= 20253)]
cvs_store = cvs_store[(cvs_store['기준_년분기_코드'] >= 20221) & (cvs_store['기준_년분기_코드'] <= 20253)]
pop_df = pop_df[(pop_df['기준_년분기_코드'] >= 20221) & (pop_df['기준_년분기_코드'] <= 20253)]

print("분석 범위 설정 완료")
print(f"  편의점 매출: {len(cvs_sales):,}건")
print(f"  편의점 점포: {len(cvs_store):,}건")
print(f"  유동인구: {len(pop_df):,}건")
print(f"  기간: {cvs_sales['기준_년분기_코드'].min()} ~ {cvs_sales['기준_년분기_코드'].max()}")

분석 범위 설정 완료
  편의점 매출: 6,097건
  편의점 점포: 6,375건
  유동인구: 6,375건
  기간: 20221 ~ 20253


## 4. 인코딩 문제 수정

일부 행정동명에 '?' 문자가 포함되어 있음 (예: 종로1?2?3?4가동)  
→ CSV 저장 시 중점(·)이 물음표로 변환된 것이므로 복원

In [15]:
# 수정 전 확인
problem_count = cvs_sales['행정동_코드_명'].str.contains('\\?', regex=True).sum()
print(f"수정 전 '?' 포함 행: {problem_count}건")

# '?' → '·' 변환
cvs_sales['행정동_코드_명'] = cvs_sales['행정동_코드_명'].str.replace('?', '·')
cvs_store['행정동_코드_명'] = cvs_store['행정동_코드_명'].str.replace('?', '·')
pop_df['행정동_코드_명'] = pop_df['행정동_코드_명'].str.replace('?', '·')
area_df['행정동_코드_명'] = area_df['행정동_코드_명'].str.replace('?', '·')

print("인코딩 수정 완료")

수정 전 '?' 포함 행: 21건
인코딩 수정 완료


## 5. 상권유형 정의

상권 구분 코드를 해석 가능한 유형으로 변환:
- A: 골목상권 (주거지역 중심)
- D: 발달상권 (상업지역 중심)
- R: 전통시장
- U: 관광특구

하나의 행정동에 여러 상권이 있을 경우, **관광특구 우선**, 없으면 **최빈값**으로 대표 상권 결정

In [16]:
# 상권유형 코드 → 명칭 변환
type_map = {'A': '골목상권', 'D': '발달상권', 'R': '전통시장', 'U': '관광특구'}
area_df['상권유형'] = area_df['상권_구분_코드'].map(type_map)

# 행정동별 대표 상권유형 결정
def get_main_type(types):
    """관광특구 우선, 없으면 최빈값"""
    if '관광특구' in types:
        return '관광특구'
    return Counter(types).most_common(1)[0][0]

area_by_dong = area_df.groupby('행정동_코드')['상권유형'].agg(list).reset_index()
area_by_dong['주요_상권유형'] = area_by_dong['상권유형'].apply(get_main_type)
area_by_dong = area_by_dong[['행정동_코드', '주요_상권유형']]

print("행정동별 상권유형 분포")
print(area_by_dong['주요_상권유형'].value_counts())

행정동별 상권유형 분포
주요_상권유형
골목상권    357
발달상권     20
전통시장     16
관광특구      6
Name: count, dtype: int64


## 6. 데이터 통합

분석 단위(행정동 × 분기)로 집계 후 병합

In [17]:
# 행정동 × 분기 단위로 집계
key_cols = ['기준_년분기_코드', '행정동_코드', '행정동_코드_명']

# 매출 집계 (종속변수)
sales_agg = cvs_sales.groupby(key_cols).agg({
    '당월_매출_금액': 'sum',
    '당월_매출_건수': 'sum'
}).reset_index()

# 점포수 집계 (독립변수 H2)
store_agg = cvs_store.groupby(key_cols).agg({
    '점포_수': 'sum'
}).reset_index()

# 유동인구 집계 (독립변수 H1)
pop_agg = pop_df.groupby(key_cols).agg({
    '총_유동인구_수': 'sum'
}).reset_index()

print(f"집계 완료: 매출 {len(sales_agg)}건, 점포 {len(store_agg)}건, 유동인구 {len(pop_agg)}건")

집계 완료: 매출 6097건, 점포 6375건, 유동인구 6375건


In [18]:
# 데이터 병합
df = sales_agg.merge(store_agg, on=key_cols, how='left')
df = df.merge(pop_agg, on=key_cols, how='left')
df = df.merge(area_by_dong, on='행정동_코드', how='left')

# 연도, 분기 컬럼 추가
df['연도'] = df['기준_년분기_코드'] // 10
df['분기'] = df['기준_년분기_코드'] % 10

print(f"통합 데이터: {len(df):,}건")

통합 데이터: 6,097건


## 7. 결측치 처리

상권영역 데이터에 없는 행정동 = 공식 지정 상권이 아닌 지역 → **'미분류'**로 처리

In [19]:
# 결측치 현황
print("결측치 현황")
print(df.isnull().sum())
print()

# 상권유형 결측 → '미분류'
n_missing = df['주요_상권유형'].isnull().sum()
df['주요_상권유형'] = df['주요_상권유형'].fillna('미분류')
print(f"상권유형 결측 {n_missing}건 → '미분류'로 처리")

# 최종 확인
print(f"\n처리 후 결측치: {df.isnull().sum().sum()}건")

결측치 현황
기준_년분기_코드      0
행정동_코드         0
행정동_코드_명       0
당월_매출_금액       0
당월_매출_건수       0
점포_수           0
총_유동인구_수       0
주요_상권유형      339
연도             0
분기             0
dtype: int64

상권유형 결측 339건 → '미분류'로 처리

처리 후 결측치: 0건


## 8. 최종 데이터 확인

가설 검증에 필요한 변수가 모두 준비되었는지 확인

In [20]:
print("=" * 60)
print("분석용 데이터 요약")
print("=" * 60)

print(f"\n[데이터 규모]")
print(f"  관측치: {len(df):,}건")
print(f"  행정동: {df['행정동_코드'].nunique()}개")
print(f"  분기: {df['기준_년분기_코드'].nunique()}개 ({df['기준_년분기_코드'].min()} ~ {df['기준_년분기_코드'].max()})")

print(f"\n[변수 구성]")
print(f"  종속변수: 당월_매출_금액")
print(f"  독립변수 H1: 총_유동인구_수")
print(f"  독립변수 H2: 점포_수")
print(f"  독립변수 H3: 주요_상권유형")

print(f"\n[상권유형 분포]")
for t, cnt in df['주요_상권유형'].value_counts().items():
    print(f"  {t}: {cnt:,}건 ({cnt/len(df)*100:.1f}%)")

분석용 데이터 요약

[데이터 규모]
  관측치: 6,097건
  행정동: 414개
  분기: 15개 (20221 ~ 20253)

[변수 구성]
  종속변수: 당월_매출_금액
  독립변수 H1: 총_유동인구_수
  독립변수 H2: 점포_수
  독립변수 H3: 주요_상권유형

[상권유형 분포]
  골목상권: 5,137건 (84.3%)
  미분류: 339건 (5.6%)
  발달상권: 300건 (4.9%)
  전통시장: 231건 (3.8%)
  관광특구: 90건 (1.5%)


In [21]:
# 데이터 샘플 확인
df.head()

,기준_년분기_코드,행정동_코드,행정동_코드_명,당월_매출_금액,당월_매출_건수,점포_수,총_유동인구_수,주요_상권유형,연도,분기
0,20221,11110515,청운효자동,"1,202,573,186",179263,4,3627519,골목상권,2022,1
1,20221,11110530,사직동,"3,761,850,443",659962,4,3402653,골목상권,2022,1
2,20221,11110540,삼청동,"1,226,802,003",138702,1,821735,발달상권,2022,1
3,20221,11110550,부암동,"607,056,495",79978,2,1283546,골목상권,2022,1
4,20221,11110560,평창동,"568,959,905",72916,3,823714,골목상권,2022,1


## 9. 데이터 저장

In [22]:
df.to_csv('./분석데이터.csv', encoding='utf-8-sig', index=False)
print("저장 완료: ./분석데이터.csv")
print(f"크기: {df.shape}")

저장 완료: ./분석데이터.csv
크기: (6097, 10)


---
## 다음 단계

**02_EDA.ipynb**에서 각 가설에 대한 탐색적 분석 수행:
- H1: 유동인구와 매출의 관계 탐색
- H2: 점포수와 매출의 관계 탐색
- H3: 상권유형별 매출 차이 탐색
- 회귀분석 가정 사전 점검